# Structs

* A *struct* assembles several values of assorted types together into a single value, so you can deal with them as a unit. 
* Given a struct, you can read and modify its individual components. 
* A struct can have methods associated with it that operate on its components.

## Named-Field Structs

### Declaring a Struct

In [2]:
// A rectangle of eight-bit grayscale pixels.
struct GrayscaleMap {
    pixels: Vec<u8>,
    size: (usize, usize)
}

### Creating a Struct Value

In [3]:
let width = 800;
let height = 600;

let image = GrayscaleMap {
    pixels: vec![0; width * height],
    size: (width, height)
};

In [4]:
fn create_image(size: (usize, usize)) -> GrayscaleMap {
    let pixels = vec![0; size.0 * size.1];
    GrayscaleMap{pixels, size} // struct expression
}

### Accessing Struct Fields

In [5]:
let image = create_image((800, 600));

assert!(image.size == (800, 600));
assert!(image.pixels.len() == image.size.0 * image.size.1);

### Visibility of Struct

* Structs are private by default, visible only in the module where they’re declared. 
* You can make a struct visible outside its module by prefixing its definition with `pub`

In [6]:
// A rectangle of eight-bit grayscale pixels.
pub struct GrayscaleMap {
    pub pixels: Vec<u8>,
    pub size: (usize, usize)
}

## Tuple-Like Structs

* The second kind of struct type is called a tuple-like struct, because it resembles a tuple:

In [7]:
struct BoundingBox(usize, usize);

let image_bbox = BoundingBox(800, 600);

assert!(image_bbox.0 * image_bbox.1 == 480_000);

The type of the variable image was redefined, so was lost.


* Individual elements of a tuple-like struct may be public or not:

In [8]:
pub struct BoundingBox(pub usize, pub usize);

### Tuple-Like Struct as Newtypes

* Tuple-like structs allow you to create types that behave like tuples but have a name and can implement traits
* This allows you to create newtypes for stricter type checking
* This is a workaround for the *orphan rule* - [New Type Pattern](https://doc.rust-lang.org/book/ch20-02-advanced-traits.html#using-the-newtype-pattern-to-implement-external-traits)

In [9]:
struct Distance(f64);
struct Velocity(f64);
struct Time(f64);

// A newtype for stricter type checking
fn calculate_distance(velocity: Velocity, time: Time) -> Distance {
    Distance(velocity.0 * time.0)
}

let velocity = Velocity(10.0);
let time = Time(5.0);
let distance = calculate_distance(velocity, time);

println!("Distance traveled: {} kilometers", distance.0);

The type of the variable image_bbox was redefined, so was lost.


Distance traveled: 50 kilometers


## Unit-Like Structs

* The third kind of struct is a little obscure: it declares a struct type with no elements at all:

In [10]:
struct Onesuch;

let o = Onesuch;

* This allows to create types that can act as tags or markers

## Defining Methods - `impl`

In [11]:
pub struct Queue {
    older: Vec<char>,
    younger: Vec<char>
}

impl Queue {
    pub fn push(&mut self, c: char) {
        self.younger.push(c);
    }

    pub fn pop(&mut self) -> Option<char> {
        if self.older.is_empty() {
            if self.younger.is_empty() {
                return None;
            }

            use std::mem::swap;
            swap(&mut self.older, &mut self.younger);
            self.older.reverse();
        }

        self.older.pop()
    }

    pub fn is_empty(&self) -> bool {
        self.older.is_empty() && self.younger.is_empty()
    }
}

In [12]:
let mut q = Queue{ older: Vec::new(), younger: Vec::new() };

q.push('@');
q.push('$');
assert!(q.pop() == Some('@'));

q.push('*');
assert!(q.pop() == Some('$'));
assert!(q.pop() == Some('*'));
assert!(q.pop() == None);

assert!(q.is_empty());

In [13]:
impl Queue {
    pub fn split(self) -> (Vec<char>, Vec<char>) {
        (self.older, self.younger)
    }
}

In [14]:
let mut q = Queue { older: Vec::new(), younger: Vec::new() };
q.push('Q');
q.push('U');
assert!(q.pop() == Some('Q'));
q.push('X');

let (older, younger) = q.split();

assert_eq!(older, vec!['U']);
assert_eq!(younger, vec!['X']);

### `new` Method

* Constructor function - doesn't take `self` as an argument (**static function**)

In [15]:
impl Queue {
    pub fn new() -> Self {
        Queue { older: Vec::new(), younger: Vec::new() }
    }
}

In [24]:
let mut q = Queue::new();

q.push('a');
q.push('b');
q.push('c');

assert_eq!(q.pop(), Some('a'));
assert_eq!(q.pop(), Some('b'));
assert_eq!(q.pop(), Some('c'));
assert_eq!(q.pop(), None);

## Generic Structs

* Generics allow you to write flexible, reusable functions and types that can work with any data type. Instead of specifying concrete types, you use type parameters, which are placeholders for types that will be specified when the function or type is used.
* Generics use parameters passed in angle brackets to specify the types they work with. For example, `fn example<T>(param: T) {}` uses `T` as a generic type parameter.

In [25]:
pub struct Stack<T> {
    items: Vec<T>
}

impl<T> Stack<T> {
    pub fn new() -> Stack<T> {
        Stack{ items: Vec::new() }
    }

    pub fn push(&mut self, item: T) {
        self.items.push(item)
    }

    pub fn pop(&mut self) -> Option<T> {
        self.items.pop()
    }

    pub fn is_empty(&self) -> bool {
        self.items.is_empty()
    }
}

* For static method calls, you can supply the type parameter explicitly using the turbofish `::<>` notation:

In [26]:
let s = Stack::<char>::new();

* But in practice, Rust can figure it out for you:

In [27]:
let mut stack_1 = Stack::new();
let mut stack_2 = Stack::new();

stack_1.push("Hello");
stack_2.push(3.1415);

stack_1.push("World");
stack_2.push(2.71);

## Deriving Common Traits for Structs

* Structs are not copyable or comparable by default
* You can derive the `Copy` and `Clone` traits for a struct type, but only if all its fields are copyable or clonable
* You can derive the `PartialEq` and `Eq` traits for a struct type, but only if all its fields are comparable for equality
* You can derive `Debug` for a struct type, but only if all its fields are printable

In [28]:
#[derive(Copy, Clone, Debug, PartialEq)]
struct Point {
    x: f64,
    y: f64
}

In [29]:
let mut pt = Point{ x: 0.0, y: 1.0 };

let other_pt = pt;
assert!(pt == other_pt);

pt.x = 42.42;
assert!(pt != other_pt);

println!("pt = {:?}", pt);
println!("other_pt = {:?}", other_pt);

pt = Point { x: 42.42, y: 1.0 }
other_pt = Point { x: 0.0, y: 1.0 }
